In [1]:
import os
import pandas as pd
import numpy as np
from math import sqrt
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [4]:
path = './data/'
data = pd.read_csv(os.path.join(path, 'xgolf_contents_final.csv'), encoding='utf-8', index_col=0)

data # 데이터 확인

,golf_name,title,id,datetime,year,month,day,time,golfscore,caddiescore,coursescore,pricescore,facility,contents
1,금강,굿 금강,yun****,2022.07.23 11:13,2022,7,23,11:13,10.0,10,10,10,10,페어웨이 그린 베스트 명품구장
2,비에이비스타퍼블릭,"양프로,티칭프로테스트를 앞둔 예비프로와 함께",Wls******,2022.07.23 11:08,2022,7,23,11:08,10.0,10,10,10,10,라운딩 체험으로 운동했는데 날씨도 햇빛 없는 날과함께 즐거웠습니다\n모든 좋았으나한...
3,몽베르,경기북부의 몽베르,NV2*******,2022.07.23 10:04,2022,7,23,10:04,10.0,10,10,10,10,편안하고 관리가 잘된 골프코스에서 라운딩했네요
4,스카이밸리,재촉만 하지 않았으면 최고였을텐데...,bab***,2022.07.22 23:11,2022,7,22,23:11,10.0,10,10,10,10,아침부터 비가 온 것을 제외하고는 좋았습니다.\n잘 관리된 페어웨이와 그린 그리고 ...
5,몽베르,몽베르라운딩후기,ljk******,2022.07.22 11:25,2022,7,22,11:25,10.0,10,10,10,10,첫홀은 비가와서 징행할까 말까 했었는데\n그래도 멀리왔는데 치고가자 고 일행이 적극...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47860,기흥,좋은골프장 좋은그린,NV5******,2018.01.01 18:46,2018,1,1,18:46,10.0,10,10,10,10,겨울에도 기흥은 그린이 좋아요.속도도 맘에듭니다
47861,아일랜드,아일랜드CC,dda****,2018.01.01 14:58,2018,1,1,14:58,10.0,10,10,10,10,바람이 좀 춥긴했지만 관리도 잘되있고 좋았습니다
47862,기흥,눈 속에서,l3l***,2018.01.01 12:13,2018,1,1,12:13,9.5,10,10,8,10,눈이 오는 바람에 정상적인 라운드를 할 수 없었지만 너무 좋은 멤버들과 캐디 덕분에...
47863,88,2017년 마지막 라운딩,pol*******,2018.01.01 04:17,2018,1,1,04:17,9.0,8,8,10,10,다행히 날씨가 춥지 않아 즐거운 라운딩이었음.\n11:32 티업. 전반 지나고 그늘...


In [3]:
data.info() # 개괄 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47828 entries, 1 to 47864
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   golf_name    47828 non-null  object 
 1   title        47828 non-null  object 
 2   id           47828 non-null  object 
 3   datetime     47828 non-null  object 
 4   year         47828 non-null  int64  
 5   month        47828 non-null  int64  
 6   day          47828 non-null  int64  
 7   time         47828 non-null  object 
 8   golfscore    47828 non-null  float64
 9   caddiescore  47828 non-null  int64  
 10  coursescore  47828 non-null  int64  
 11  pricescore   47828 non-null  int64  
 12  facility     47828 non-null  int64  
 13  contents     47828 non-null  object 
dtypes: float64(1), int64(7), object(6)
memory usage: 5.5+ MB


### 데이터 전처리

In [4]:
# 중복 행 확인
data[(data.duplicated())]

,golf_name,title,id,datetime,year,month,day,time,golfscore,caddiescore,coursescore,pricescore,facility,contents
82,그린힐,최고의 캐디분을 만난 날,don*******,2022.07.04 12:26,2022,7,4,12:26,8.5,10,8,10,6,폭염으로 아주 힘든 라운딩였지만 그래도 캐디님의 밝은 웃음과 성실한 태도에 정말 감...
303,샴발라,중간이상,kaa***,2022.05.26 09:15,2022,5,26,09:15,6.5,10,8,4,4,코스가 어렵지만 재미있었어요
1422,서원힐스,겨울골프,kin****,2022.02.04 09:44,2022,2,4,09:44,9.0,10,10,8,8,올해 첫 라운딩. 부킹할 때만해도 조금은 덜 추우리라 예상했지만 역시 겨울은 겨울....
2385,스카이밸리,？？,yh0***,2021.09.12 11:05,2021,9,12,11:05,9.5,10,8,10,10,코스 관리 최고 그린 관리가 조금 아쉽네요 ??
3091,강남300,다 좋았네요,KK1*********,2021.07.31 13:59,2021,7,31,13:59,10.0,10,10,10,10,가격대비 관리도 잘 되어있고 ??
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47860,기흥,좋은골프장 좋은그린,NV5******,2018.01.01 18:46,2018,1,1,18:46,10.0,10,10,10,10,겨울에도 기흥은 그린이 좋아요.속도도 맘에듭니다
47861,아일랜드,아일랜드CC,dda****,2018.01.01 14:58,2018,1,1,14:58,10.0,10,10,10,10,바람이 좀 춥긴했지만 관리도 잘되있고 좋았습니다
47862,기흥,눈 속에서,l3l***,2018.01.01 12:13,2018,1,1,12:13,9.5,10,10,8,10,눈이 오는 바람에 정상적인 라운드를 할 수 없었지만 너무 좋은 멤버들과 캐디 덕분에...
47863,88,2017년 마지막 라운딩,pol*******,2018.01.01 04:17,2018,1,1,04:17,9.0,8,8,10,10,다행히 날씨가 춥지 않아 즐거운 라운딩이었음.\n11:32 티업. 전반 지나고 그늘...


In [5]:
# 중복 행 (18073) 제거
data = data.drop_duplicates()
data

,golf_name,title,id,datetime,year,month,day,time,golfscore,caddiescore,coursescore,pricescore,facility,contents
1,금강,굿 금강,yun****,2022.07.23 11:13,2022,7,23,11:13,10.0,10,10,10,10,페어웨이 그린 베스트 명품구장
2,비에이비스타퍼블릭,"양프로,티칭프로테스트를 앞둔 예비프로와 함께",Wls******,2022.07.23 11:08,2022,7,23,11:08,10.0,10,10,10,10,라운딩 체험으로 운동했는데 날씨도 햇빛 없는 날과함께 즐거웠습니다\n모든 좋았으나한...
3,몽베르,경기북부의 몽베르,NV2*******,2022.07.23 10:04,2022,7,23,10:04,10.0,10,10,10,10,편안하고 관리가 잘된 골프코스에서 라운딩했네요
4,스카이밸리,재촉만 하지 않았으면 최고였을텐데...,bab***,2022.07.22 23:11,2022,7,22,23:11,10.0,10,10,10,10,아침부터 비가 온 것을 제외하고는 좋았습니다.\n잘 관리된 페어웨이와 그린 그리고 ...
5,몽베르,몽베르라운딩후기,ljk******,2022.07.22 11:25,2022,7,22,11:25,10.0,10,10,10,10,첫홀은 비가와서 징행할까 말까 했었는데\n그래도 멀리왔는데 치고가자 고 일행이 적극...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38084,기흥,좋은골프장 좋은그린,NV5******,2018.01.01 18:46,2018,1,1,18:46,10.0,10,10,10,10,겨울에도 기흥은 그린이 좋아요.속도도 맘에듭니다
38085,아일랜드,아일랜드CC,dda****,2018.01.01 14:58,2018,1,1,14:58,10.0,10,10,10,10,바람이 좀 춥긴했지만 관리도 잘되있고 좋았습니다
38086,기흥,눈 속에서,l3l***,2018.01.01 12:13,2018,1,1,12:13,9.5,10,10,8,10,눈이 오는 바람에 정상적인 라운드를 할 수 없었지만 너무 좋은 멤버들과 캐디 덕분에...
38087,88,2017년 마지막 라운딩,pol*******,2018.01.01 04:17,2018,1,1,04:17,9.0,8,8,10,10,다행히 날씨가 춥지 않아 즐거운 라운딩이었음.\n11:32 티업. 전반 지나고 그늘...


In [6]:
# 같은 아이디로 같은 제목과 내용의 글을 올린 경우 있는지 확인
data_dup = data[(data.duplicated(['id', 'title', 'contents']))]
data_dup

,golf_name,title,id,datetime,year,month,day,time,golfscore,caddiescore,coursescore,pricescore,facility,contents
81,그린힐,최고의 캐디분을 만난 날,don*******,2022.07.04 12:26,2022,7,4,12:26,8.5,10,8,10,6,폭염으로 아주 힘든 라운딩였지만 그래도 캐디님의 밝은 웃음과 성실한 태도에 정말 감...
7245,레이크사이드,좋아요,kei*****,2020.10.04 11:22,2020,10,4,11:22,10.0,10,10,10,10,역시 레이크사이드♥?
15928,자유,자유cc 후기,khl****,2019.03.07 21:12,2019,3,7,21:12,9.0,10,8,10,8,평일오후 선결제로 저렴하게 다녀왔어요.\n겨울이 끝나가는데 날씨도 좋고 밀리지도 않...
16035,레이크힐스안성(P9),가성비굿,kks*****,2019.03.03 20:28,2019,3,3,20:28,10.0,10,10,10,10,가격대비 아주좋습니다
16772,88,88,act*****,2018.12.17 15:07,2018,12,17,15:07,10.0,10,10,10,10,너무 좋아요 최고에요
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37022,아일랜드,잘치고ㅡ,lhy****,2018.03.06 11:11,2018,3,6,11:11,10.0,10,10,10,10,날씨도넘좋았고ㅡ동반자들도좋아했다ㅡ\n나중에또오고십다는말을
37216,블루헤런,블루헤런을 다녀와서~^^,oza**,2018.03.02 11:11,2018,3,2,11:11,10.0,10,10,10,10,가성비날씨그리고 캐디까지 친절해서 즐거운 롼딩이였네요 ^^
37266,써닝포인트,최악이었네요,NV4*******,2018.03.01 10:06,2018,3,1,10:06,7.5,8,6,8,8,캐디가 착해서 그냥 재밌게치고 오자했지만\n시작부터 티업시간도 되지 않았는데 짜증나...
37276,남서울,코스관리 잘 되어있는 골프장,plo***,2018.02.28 22:25,2018,2,28,22:25,10.0,10,10,10,10,캐디도 좋았고 코스도 좋았습니딘


In [7]:
# 위 경우 (103) 리뷰 신뢰도가 낮을것으로 판단 => 삭제
data = data.drop_duplicates(['id', 'title', 'contents'])
data.shape

(29652, 14)

In [8]:
# 중복 행 확인
data[(data.duplicated())]

,golf_name,title,id,datetime,year,month,day,time,golfscore,caddiescore,coursescore,pricescore,facility,contents


In [9]:
data['id'].nunique() # unique id 개수 확인

6134

In [10]:
data['id'].value_counts() # id별 리뷰 개수 확인(소셜로그인 계정 - NV: Naver, KK: Kakao)

NV2*******      889
KK1*********    808
NV3*******      685
NV1*******      549
NV4*******      479
               ... 
Woo***            1
skk******         1
nk8***            1
hwl****           1
pac*****          1
Name: id, Length: 6134, dtype: int64

In [11]:
# 네이버 계정 사용 리뷰 확인
data[data['id'].str.contains("NV", case=True)]

,golf_name,title,id,datetime,year,month,day,time,golfscore,caddiescore,coursescore,pricescore,facility,contents
3,몽베르,경기북부의 몽베르,NV2*******,2022.07.23 10:04,2022,7,23,10:04,10.0,10,10,10,10,편안하고 관리가 잘된 골프코스에서 라운딩했네요
15,서서울,언제나좋은 서서울,NV4*******,2022.07.20 11:19,2022,7,20,11:19,9.5,10,10,10,8,음식가격비싼거만빼면 너무좋은 코스관리등
19,크리스탈밸리,칭찬합니다.,NV9*******,2022.07.19 15:43,2022,7,19,15:43,8.5,10,8,8,8,더운 날씨\n짜증 한 번 없이 열심히 해 준\n캐디 ?혜원 양 칭찬합니다.
22,360도,귿,NV5*******,2022.07.18 11:10,2022,7,18,11:10,10.0,10,10,10,10,귿 또 방문할 예정입니다!
23,서서울,서서울cc,NV1*******,2022.07.18 11:08,2022,7,18,11:08,7.5,6,10,8,6,그린피가 비싼만큼 코스관리는 훌륭했습니다\n다만 캐디님이 신입이셔서.... 캐디피가...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38056,레이크사이드,겨울 추운 날,NV_********,2018.01.03 16:38,2018,1,3,16:38,9.0,6,10,10,10,코스는 얼었어도 재미있음 영상기온을 기다리며~~~
38057,기흥,기흥cc 후기,NV2*******,2018.01.03 15:28,2018,1,3,15:28,8.0,8,8,8,8,12월 30일에 갔는데 날씨도 따뜻해서 좋았습니다\n그린이 얼어서 볼이 튀기도 하고...
38082,솔모로,2017마지막 라운딩,NV2*******,2018.01.01 21:20,2018,1,1,21:20,10.0,10,10,10,10,12/29일 진행한 2인 라운딩\n그주에 가장 따뜻한 하루여서 아주아주 행복하게 게...
38083,은화삼,겨울골프 중 최고,NV8******,2018.01.01 20:24,2018,1,1,20:24,10.0,10,10,10,10,최고의 날씨에서 즐거운 란딩했어요


In [12]:
# 카카오 계정 사용 리뷰 확인
data[data['id'].str.contains("KK", case=True)]

,golf_name,title,id,datetime,year,month,day,time,golfscore,caddiescore,coursescore,pricescore,facility,contents
8,이포,이포CC 좋아요,KK2*********,2022.07.21 13:06,2022,7,21,13:06,10.0,10,10,10,10,아늑하고 편안한 라운딩 되었어요\n\n재방문 의사 있어요
17,포레스트힐(P6),연습 겸 2인 라운딩,KK1*********,2022.07.19 15:57,2022,7,19,15:57,10.0,10,10,10,10,노캐디\n연습 2인 라운딩 잘 다녀왔습니다\n가깝고 저렴하고 좋아요
26,중부,"필드는 명문, 캐디는 최악",KK1*********,2022.07.18 07:24,2022,7,18,07:24,8.0,2,10,10,10,역시나 필드 관리나 클럽하우스 시설은 최고로 좋았지만..\n캐디 때문에 완전 기분 ...
27,이포,최악 이포,KK1*********,2022.07.17 14:36,2022,7,17,14:36,4.0,2,8,4,2,"장점 : 근거리, 넓은 페어웨이, 잔디 상태\n단점 : 최악 캐디(고령, 불친절, ..."
29,해솔리아,굿,KK2*********,2022.07.17 11:21,2022,7,17,11:21,10.0,10,10,10,10,인터넷 이용자 댓글보고 기대도 안했는데 아두 만족함
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10263,한림광릉,굿..,KK1*********,2020.04.16 11:27,2020,4,16,11:27,10.0,10,10,10,10,부킹을쉽게할수있어서조앗구\n시설두아주깔끔하구잔디상태완전조앗습니다.
10612,크리스탈밸리,날씨도 좋고 기분좋게 다녀왓어요~!!,KK1*********,2020.03.24 18:25,2020,3,24,18:25,8.5,8,8,8,10,크리스탈밸리 너무 기분좋게 다녀왔어요ㅎㅎ\n날씨도 좋고 클럽하우스 음식도 너무 맛있...
11310,서원힐스,서원 야간 다녀왔습니다~^^,KK1*********,2019.11.21 00:00,2019,11,21,00:00,9.0,8,10,8,10,야간 라운딩 했어요~\n날씨도 좋고 코스도 좋고 즐거운 란딩했습니다~
13244,서원힐스,골뱅이 무침이 짱이에요~,KK1*********,2019.07.11 00:00,2019,7,11,00:00,9.5,8,10,10,10,그늘집에서 골뱅이 무침이랑 막걸리 한잔하면 크~ 홀인원 할수있을 정도로 최곱니다 ㅎㅎ


In [13]:
# 소셜로그인 계정 (4010 + 848) 리뷰 삭제(과도한 중복 피하기) => 행 개수: 29652 - (4010 + 848) = 24794
data = data[~((data['id'].str.contains('NV', case=True)) | (data['id'].str.contains('KK', case=True)))]
data.shape

(24794, 14)

In [14]:
data[(data.duplicated())]

,golf_name,title,id,datetime,year,month,day,time,golfscore,caddiescore,coursescore,pricescore,facility,contents


In [15]:
# Base DF 생성
data.rename(columns={'golf_name':'cc_name', 'golfscore':'cc_score', 'caddiescore':'caddie', 'coursescore':'course', 'pricescore': 'price'}, inplace=True)
print(data.shape)
data.head()

(24794, 14)


,cc_name,title,id,datetime,year,month,day,time,cc_score,caddie,course,price,facility,contents
1,금강,굿 금강,yun****,2022.07.23 11:13,2022,7,23,11:13,10.0,10,10,10,10,페어웨이 그린 베스트 명품구장
2,비에이비스타퍼블릭,"양프로,티칭프로테스트를 앞둔 예비프로와 함께",Wls******,2022.07.23 11:08,2022,7,23,11:08,10.0,10,10,10,10,라운딩 체험으로 운동했는데 날씨도 햇빛 없는 날과함께 즐거웠습니다\n모든 좋았으나한...
4,스카이밸리,재촉만 하지 않았으면 최고였을텐데...,bab***,2022.07.22 23:11,2022,7,22,23:11,10.0,10,10,10,10,아침부터 비가 온 것을 제외하고는 좋았습니다.\n잘 관리된 페어웨이와 그린 그리고 ...
5,몽베르,몽베르라운딩후기,ljk******,2022.07.22 11:25,2022,7,22,11:25,10.0,10,10,10,10,첫홀은 비가와서 징행할까 말까 했었는데\n그래도 멀리왔는데 치고가자 고 일행이 적극...
6,썬힐,정말 해도해도 너무하네요,cli*****,2022.07.22 09:30,2022,7,22,09:30,3.5,8,2,2,2,좋은 날 기분 더럽게 다녀왔네요.. 코스관리 개엉망이고 그린은 완전 걸레짝. 카운터...


### 사용자 데이터프레임 생성

In [16]:
# 전처리 후 unique id 개수 확인
data['id'].nunique()

6111

In [17]:
# 사용자 일련번호를 부여한 USER DF 생성
data_id = data['id'].sort_values().drop_duplicates().reset_index()
data_id = data_id.rename(columns={'index':'id_num'})
data_id['id_num'] = range(1,len(data_id)+1)
data_id

,id_num,id
0,1,002***
1,2,003****
2,3,007***
3,4,007*****
4,5,010*******
...,...,...
6106,6107,zzi****
6107,6108,zzi*********
6108,6109,zzu****
6109,6110,zzz*


### 골프장 데이터프레임 생성

In [18]:
# 골프장 DF 생성
data_cc = data['cc_name'].sort_values().drop_duplicates().reset_index()
data_cc = data_cc.rename(columns={'index':'cc_num'})
data_cc['cc_num'] = range(1,len(data_cc)+1)
data_cc

,cc_num,cc_name
0,1,360도
1,2,88
2,3,H1 CLUB(구 덕평)
3,4,XGOLF
4,5,XGOLF(테스트)
...,...,...
134,135,한양파인(P9)
135,136,한원
136,137,해솔리아
137,138,화성(P9)


In [19]:
data_cc.head(10)

,cc_num,cc_name
0,1,360도
1,2,88
2,3,H1 CLUB(구 덕평)
3,4,XGOLF
4,5,XGOLF(테스트)
5,6,가산노블리스
6,7,가평베네스트
7,8,강남300
8,9,고양컨트리클럽(P9)
9,10,곤지암


In [20]:
# XGOLF는 골프장이 아닌 테스트용 인것으로 보임
data_test = data[(data['cc_name'].str.contains('XGOLF'))]
data_test.shape

(27, 14)

In [21]:
# 추가 전처리 필요... XGOLF 삭제하여 Base DF 변경 -> 행 개수: 24794 - 27 = 24767
data = data[~(data['cc_name'].str.contains('XGOLF'))]
data.shape

(24767, 14)

In [22]:
# Base DF 변경에 따른 USER DF 재생성
data_id = data['id'].sort_values().drop_duplicates().reset_index()
data_id = data_id.rename(columns={'index':'id_num'})
data_id['id_num'] = range(1,len(data_id)+1)
data_id

,id_num,id
0,1,002***
1,2,003****
2,3,007***
3,4,007*****
4,5,010*******
...,...,...
6103,6104,zzi****
6104,6105,zzi*********
6105,6106,zzu****
6106,6107,zzz*


In [23]:
# id 정보 csv 추출
# data_id.to_csv('./data/xgolf_user_completed.csv')

In [24]:
# Base DF 변경에 따른 골프장 DF 재생성
data_cc = data['cc_name'].sort_values().drop_duplicates().reset_index()
data_cc = data_cc.rename(columns={'index':'cc_num'})
data_cc['cc_num'] = range(1,len(data_cc)+1)
data_cc

,cc_num,cc_name
0,1,360도
1,2,88
2,3,H1 CLUB(구 덕평)
3,4,가산노블리스
4,5,가평베네스트
...,...,...
132,133,한양파인(P9)
133,134,한원
134,135,해솔리아
135,136,화성(P9)


In [25]:
# 골프장 정보 csv 추출
# data_cc.to_csv('./data/xgolf_cc_completed.csv')

In [26]:
# Base DF에 id_num 및 cc_num 삽입
data = pd.merge(data, data_id, on = 'id', how = 'left')
data = pd.merge(data, data_cc, on = 'cc_name', how = 'left')
data

,cc_name,title,id,datetime,year,month,day,time,cc_score,caddie,course,price,facility,contents,id_num,cc_num
0,금강,굿 금강,yun****,2022.07.23 11:13,2022,7,23,11:13,10.0,10,10,10,10,페어웨이 그린 베스트 명품구장,6029,15
1,비에이비스타퍼블릭,"양프로,티칭프로테스트를 앞둔 예비프로와 함께",Wls******,2022.07.23 11:08,2022,7,23,11:08,10.0,10,10,10,10,라운딩 체험으로 운동했는데 날씨도 햇빛 없는 날과함께 즐거웠습니다\n모든 좋았으나한...,186,53
2,스카이밸리,재촉만 하지 않았으면 최고였을텐데...,bab***,2022.07.22 23:11,2022,7,22,23:11,10.0,10,10,10,10,아침부터 비가 온 것을 제외하고는 좋았습니다.\n잘 관리된 페어웨이와 그린 그리고 ...,460,68
3,몽베르,몽베르라운딩후기,ljk******,2022.07.22 11:25,2022,7,22,11:25,10.0,10,10,10,10,첫홀은 비가와서 징행할까 말까 했었는데\n그래도 멀리왔는데 치고가자 고 일행이 적극...,3498,44
4,썬힐,정말 해도해도 너무하네요,cli*****,2022.07.22 09:30,2022,7,22,09:30,3.5,8,2,2,2,좋은 날 기분 더럽게 다녀왔네요.. 코스관리 개엉망이고 그린은 완전 걸레짝. 카운터...,887,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24762,스카이밸리,마운틴코스라기에. . .,ana*****,2018.01.01 21:50,2018,1,1,21:50,9.5,10,8,10,10,눈이라도 많을까봐 걱정했는데.\n플레이하기에 전혀 지장이 없는\n상태로 코스는 잘 ...,333,68
24763,아일랜드,아일랜드CC,dda****,2018.01.01 14:58,2018,1,1,14:58,10.0,10,10,10,10,바람이 좀 춥긴했지만 관리도 잘되있고 좋았습니다,1097,78
24764,기흥,눈 속에서,l3l***,2018.01.01 12:13,2018,1,1,12:13,9.5,10,10,8,10,눈이 오는 바람에 정상적인 라운드를 할 수 없었지만 너무 좋은 멤버들과 캐디 덕분에...,3355,16
24765,88,2017년 마지막 라운딩,pol*******,2018.01.01 04:17,2018,1,1,04:17,9.0,8,8,10,10,다행히 날씨가 춥지 않아 즐거운 라운딩이었음.\n11:32 티업. 전반 지나고 그늘...,4386,2


In [27]:
# 데이터 병합 확인
print(data['id_num'].isnull().sum(), data['cc_num'].isnull().sum())

0 0


In [28]:
# 혹시 모를 중복 확인
data[(data.duplicated())]

,cc_name,title,id,datetime,year,month,day,time,cc_score,caddie,course,price,facility,contents,id_num,cc_num


In [29]:
# 전처리 끝난 최종 데이터 csv 추출
# data.to_csv('./data/xgolf_contents_completed.csv')

In [2]:
path = './data/'
data = pd.read_csv(os.path.join(path, 'xgolf_contents_completed.csv'), encoding='utf-8', index_col=0)
data_id = pd.read_csv(os.path.join(path, 'xgolf_user_completed.csv'), encoding='utf-8', index_col=0)
data_cc = pd.read_csv(os.path.join(path, 'xgolf_cc_completed.csv'), encoding='utf-8', index_col=0)
ratings_cc = pd.read_csv(os.path.join(path, 'overall_rating_completed.csv'), encoding='utf-8', index_col=0)
ratings_caddie = pd.read_csv(os.path.join(path, 'caddie_rating_completed.csv'), encoding='utf-8', index_col=0)
ratings_course = pd.read_csv(os.path.join(path, 'course_rating_completed.csv'), encoding='utf-8', index_col=0)
ratings_price = pd.read_csv(os.path.join(path, 'price_rating_completed.csv'), encoding='utf-8', index_col=0)
ratings_facility = pd.read_csv(os.path.join(path, 'facility_rating_completed.csv'), encoding='utf-8', index_col=0)


In [6]:
# ratings_cc_df = data[['id_num', 'cc_num', 'cc_score', 'cc_name']]
# ratings_caddie_df = data[['id_num', 'cc_num', 'caddie', 'cc_name']]
# ratings_course_df = data[['id_num', 'cc_num', 'course', 'cc_name']]
# ratings_price_df = data[['id_num', 'cc_num', 'price', 'cc_name']]
# ratings_facility_df = data[['id_num', 'cc_num', 'facility', 'cc_name']]

# ratings_cc_df.to_csv('./data/overall_rating_completed.csv')
# ratings_caddie_df.to_csv('./data/caddie_rating_completed.csv')
# ratings_course_df.to_csv('./data/course_rating_completed.csv')
# ratings_price_df.to_csv('./data/price_rating_completed.csv')
# ratings_facility_df.to_csv('./data/facility_rating_completed.csv')

## CF - KNN

In [31]:
# # 우선 필요한 데이터만 발췌
# ratings_cc_df = data[['id_num', 'cc_num', 'cc_score']]
# ratings_cc_df

,id_num,cc_num,cc_score
0,6029,15,10.0
1,186,53,10.0
2,460,68,10.0
3,3498,44,10.0
4,887,73,3.5
...,...,...,...
24762,333,68,9.5
24763,1097,78,10.0
24764,3355,16,9.5
24765,4386,2,9.0


In [10]:
# null 체크
ratings_cc.isnull().sum()

id_num      0
cc_num      0
cc_score    0
cc_name     0
dtype: int64

In [8]:
# # cc_name 컬럼을 얻기 위해 조인 수행
# rating_cc_info = pd.merge(ratings_cc_df, data_cc, on='cc_num')
# rating_cc_info

In [15]:
# pivot_table 메소드를 사용해서 행렬 변환
ratings_cc_matrix = ratings_cc.pivot_table('cc_score', index='id_num', columns='cc_name', aggfunc='min')
ratings_caddie_matrix = ratings_caddie.pivot_table('caddie', index='id_num', columns='cc_name', aggfunc='min')
ratings_course_matrix = ratings_course.pivot_table('course', index='id_num', columns='cc_name', aggfunc='min')
ratings_price_matrix = ratings_price.pivot_table('price', index='id_num', columns='cc_name', aggfunc='min')
ratings_facility_matrix = ratings_facility.pivot_table('facility', index='id_num', columns='cc_name', aggfunc='min')

print(ratings_cc_matrix.shape, ratings_caddie_matrix.shape, ratings_course_matrix.shape, ratings_price_matrix.shape, ratings_facility_matrix.shape, sep='\n')

(6108, 137)
(6108, 137)
(6108, 137)
(6108, 137)
(6108, 137)


In [ ]:
# # columns='title' 로 title 컬럼으로 pivot 수행. 
# ratings_matrix = rating_cc_info.pivot_table('cc_score', index='id_num', columns='cc_name', aggfunc=max)
# ratings_matrix

In [14]:
# NaN 값을 모두 0 으로 변환
ratings_matrix = ratings_cc_matrix.fillna(0)
ratings_matrix

cc_name,360도,88,H1 CLUB(구 덕평),가산노블리스,가평베네스트,강남300,고양컨트리클럽(P9),곤지암,골드,골프존카운티안성H,...,필로스,한림광릉,한림용인,한성,한양,한양파인(P9),한원,해솔리아,화성(P9),화성상록
id_num,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# 아이템-사용자 행렬로 transpose 한다.
ratings_matrix_T = ratings_matrix.transpose()    # 전치 행렬

print(ratings_matrix_T.shape)
ratings_matrix_T.head(5)

(137, 6108)


id_num,1,2,3,4,5,6,7,8,9,10,...,6099,6100,6101,6102,6103,6104,6105,6106,6107,6108
cc_name,,,,,,,,,,,,,,,,,,,,,
360도,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
H1 CLUB(구 덕평),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
가산노블리스,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
가평베네스트,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 코사인 유사도

In [16]:
# 골프장들 간 코사인 유사도 산출
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity()로 반환된 넘파이 행렬을 골프장에 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df

(137, 137)


cc_name,360도,88,H1 CLUB(구 덕평),가산노블리스,가평베네스트,강남300,고양컨트리클럽(P9),곤지암,골드,골프존카운티안성H,...,필로스,한림광릉,한림용인,한성,한양,한양파인(P9),한원,해솔리아,화성(P9),화성상록
cc_name,,,,,,,,,,,,,,,,,,,,,
360도,1.000000,0.127315,0.083301,0.0,0.0,0.080223,0.0,0.076589,0.089323,0.051565,...,0.119028,0.108239,0.022208,0.036934,0.020085,0.015109,0.063174,0.070750,0.000000,0.021922
88,0.127315,1.000000,0.121103,0.0,0.0,0.165174,0.0,0.046346,0.152532,0.088981,...,0.061556,0.137236,0.040934,0.153491,0.030233,0.049017,0.053266,0.031397,0.026264,0.045146
H1 CLUB(구 덕평),0.083301,0.121103,1.000000,0.0,0.0,0.106111,0.0,0.097707,0.120663,0.105919,...,0.082755,0.089859,0.054910,0.120163,0.043559,0.065348,0.053949,0.025347,0.037731,0.015382
가산노블리스,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
가평베네스트,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.074358,0.041324,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
한양파인(P9),0.015109,0.049017,0.065348,0.0,0.0,0.072396,0.0,0.000000,0.056063,0.033312,...,0.056003,0.047626,0.039003,0.014750,0.026684,1.000000,0.063751,0.000000,0.016251,0.042349
한원,0.063174,0.053266,0.053949,0.0,0.0,0.064732,0.0,0.000000,0.022292,0.040795,...,0.019949,0.028375,0.102120,0.058563,0.000000,0.063751,1.000000,0.043373,0.052212,0.000000
해솔리아,0.070750,0.031397,0.025347,0.0,0.0,0.042029,0.0,0.000000,0.087234,0.058366,...,0.025494,0.043039,0.044781,0.051965,0.000000,0.000000,0.043373,1.000000,0.021170,0.000000


In [17]:
# 가평베네스트와 유사한 골프장 5개 확인해보기
item_sim_df['가평베네스트'].sort_values(ascending=False)[1:6]

cc_name
파인크리크      0.196455
일동레이크      0.169238
사우스스프링스    0.084848
프리스틴밸리     0.083111
자유         0.082756
Name: 가평베네스트, dtype: float64

### 개인화 된 골프장 추천

In [18]:
# 평점 벡터(행 벡터)와 유사도 벡터(열 벡터)를 내적(dot)해서 예측 평점을 계산하는 함수 정의
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)

# 데이터프레임으로 변환
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

In [19]:
print(ratings_pred_matrix.shape)
ratings_pred_matrix

(6108, 137)


cc_name,360도,88,H1 CLUB(구 덕평),가산노블리스,가평베네스트,강남300,고양컨트리클럽(P9),곤지암,골드,골프존카운티안성H,...,필로스,한림광릉,한림용인,한성,한양,한양파인(P9),한원,해솔리아,화성(P9),화성상록
id_num,,,,,,,,,,,,,,,,,,,,,
1,0.126661,0.154646,0.097587,0.0,0.195213,0.135697,0.000000,0.000000,0.101452,0.117260,...,0.103588,0.123115,0.084206,0.136896,0.178283,0.134735,0.088635,0.113092,0.097834,0.180032
2,0.273082,0.290221,0.250303,0.0,0.118620,0.292067,0.000000,0.178344,0.295361,0.306964,...,0.206585,0.273211,0.187045,0.302079,0.146161,0.160645,0.177738,0.294799,0.094308,0.168462
3,0.120058,0.177105,0.154216,0.0,0.000000,0.145958,0.000000,0.000000,0.204272,1.419705,...,0.092665,0.175591,0.124618,0.189150,0.180221,0.150423,0.164096,0.169523,0.097216,0.126546
4,0.159005,0.094831,0.113135,0.0,0.000000,0.089186,0.000000,0.000000,0.083292,0.093975,...,0.092211,0.102997,0.088126,0.086223,0.012836,0.087934,0.125270,0.123201,0.048289,0.000000
5,0.043909,0.052111,0.055159,0.0,0.068170,0.057105,0.000000,0.000000,0.046626,0.041743,...,0.080176,0.088154,0.042571,0.047899,0.073701,0.043533,0.032281,0.025122,0.016950,0.014927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,0.101378,0.141091,0.130916,0.0,0.000000,0.143767,0.000000,0.113917,0.956408,0.146018,...,0.078835,0.118700,0.130237,0.103266,0.129292,0.098784,0.038930,0.161775,0.086516,0.031400
6105,0.138614,0.099928,0.097925,0.0,0.000000,0.098879,0.000000,0.000000,0.114625,0.074893,...,0.120507,0.121239,0.118218,0.085108,0.026938,0.032728,0.064504,0.124658,0.099912,0.088750
6106,0.138018,0.223201,0.189773,0.0,0.212653,0.234922,0.000000,0.000000,0.180819,0.134892,...,0.142438,0.218108,0.140986,0.277556,0.088055,0.134180,0.204248,0.143999,0.155821,0.138515


In [20]:
# 사용자가 평점을 부여한 골프장에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 인접 이웃 MSE:  51.40056886949116


### top-n 유사도를 가진 데이터들에 대해서만 예측 평점 계산
- MSE 값을 감소하기 위해 특정 골프장과 가장 비슷한 유사도를 가지는 골프장에 대해서만 유사도 벡터를 적용

In [21]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))        
    return pred

In [22]:
# 실행시간 2분 정도 걸림
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values))

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
ratings_pred_matrix

/var/folders/dz/4s3j51857zn3t537wl7_r8th0000gn/T/ipykernel_1051/2394360531.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
/var/folders/dz/4s3j51857zn3t537wl7_r8th0000gn/T/ipykernel_1051/2394360531.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))


아이템 기반 인접 TOP-20 이웃 MSE:  29.962951426751125


cc_name,360도,88,H1 CLUB(구 덕평),가산노블리스,가평베네스트,강남300,고양컨트리클럽(P9),곤지암,골드,골프존카운티안성H,...,필로스,한림광릉,한림용인,한성,한양,한양파인(P9),한원,해솔리아,화성(P9),화성상록
id_num,,,,,,,,,,,,,,,,,,,,,
1,0.343310,0.375261,0.000000,0.0,0.198051,0.343177,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.346878,0.00000,0.300651,0.271328,0.268908,0.000000,0.000000,0.000000,0.292445
2,0.297899,0.328964,0.309845,0.0,0.120345,0.314239,0.000000,0.187866,0.617075,0.601112,...,0.000000,0.315593,0.00000,0.257814,0.000000,0.000000,0.000000,0.237027,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.426837,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.205563
4,0.430978,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.237007,0.265606,0.000000,0.000000
5,0.000000,0.000000,0.150390,0.0,0.069161,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.208033,0.248375,0.00000,0.000000,0.112166,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,0.000000,0.342369,0.356938,0.0,0.000000,0.363586,0.000000,0.119999,2.519162,0.352454,...,0.000000,0.000000,0.27717,0.000000,0.000000,0.000000,0.000000,0.348766,0.000000,0.000000
6105,0.375710,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.312678,0.341590,0.00000,0.000000,0.000000,0.000000,0.000000,0.268747,0.000000,0.000000
6106,0.000000,0.304950,0.293837,0.0,0.215744,0.368315,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.362590,0.00000,0.609566,0.000000,0.000000,0.242043,0.000000,0.197772,0.000000


In [23]:
# 추천에 앞서 2번 사용자가 높은 평점을 준 골프장을 확인
user_rating_id = ratings_matrix.loc[2, :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:5]

cc_name
아리지    10.0
금강      8.0
그린힐     7.5
Name: 2, dtype: float64

#### 사용자가 방문하지 않은 골프장 중에서 추천해보자
- user_rating이 0보다 크면 기존에 방문한 골프장이라는 점을 이용해서 계산

In [4]:
def get_unseen_cc(ratings_matrix, id_num):
    # id_num으로 입력받은 사용자의 모든 골프장 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 골프장명(cc_name)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[id_num,:]
    
    # user_rating이 0보다 크면 기존에 방문한 골프장. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[user_rating > 0].index.tolist()
    
    # 모든 골프장을 list 객체로 만듬.
    cc_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 cc는 cc_list에서 제외함. 
    unseen_list = [cc for cc in cc_list if cc not in already_seen]
    
    return unseen_list

In [5]:
# pred_df : 앞서 계산된 골프장별 예측 평점
# unseen_list : 사용자가 방문하지 않은 골프장
# top_n : 상위 n개를 가져온다.

def recomm_cc_by_id(pred_df, id_num, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 골프장명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_cc = pred_df.loc[id_num, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_cc

In [26]:
# 사용자가 방문하지 않은 골프장 추출   
unseen_list = get_unseen_cc(ratings_matrix, 2)

# 아이템 기반의 인접 이웃 협업 필터링으로 골프장 추천 
recomm_10_cc = recomm_cc_by_id(ratings_pred_matrix, 2, unseen_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_10_cc = pd.DataFrame(data=recomm_10_cc.values, index=recomm_10_cc.index, columns=['pred_score'])
recomm_10_cc

,pred_score
cc_name,
안성,0.795842
여주,0.759187
에덴블루,0.726302
캐슬파인,0.707528
루나힐스안성,0.692187
타이거,0.650705
큐로,0.647471
리베라,0.640060
골드,0.617075


#### LJW style

In [16]:
# train_df, test_df = train_test_split(ratings_cc_df, test_size=0.02, random_state=123)

# print(train_df.shape)
# print(test_df.shape)

sparse_matrix = ratings_cc.pivot_table('cc_score', index='id_num', columns='cc_name', aggfunc='min')
sparse_matrix

cc_name,360도,88,H1 CLUB(구 덕평),가산노블리스,가평베네스트,강남300,고양컨트리클럽(P9),곤지암,골드,골프존카운티안성H,...,필로스,한림광릉,한림용인,한성,한양,한양파인(P9),한원,해솔리아,화성(P9),화성상록
id_num,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
item_sparse_matrix = sparse_matrix.T.fillna(0)
item_sparse_matrix

id_num,1,2,3,4,5,6,7,8,9,10,...,6099,6100,6101,6102,6103,6104,6105,6106,6107,6108
cc_name,,,,,,,,,,,,,,,,,,,,,
360도,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
H1 CLUB(구 덕평),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
가산노블리스,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
가평베네스트,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
한양파인(P9),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
한원,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
해솔리아,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
def cossim_matrix(a, b):
    cossim_values = cosine_similarity(a.values, b.values)
    cossim_df = pd.DataFrame(data=cossim_values, columns = a.index.values, index=a.index)

    return cossim_df

In [30]:
item_cossim_df = cossim_matrix(item_sparse_matrix, item_sparse_matrix)
item_cossim_df

,360도,88,H1 CLUB(구 덕평),가산노블리스,가평베네스트,강남300,고양컨트리클럽(P9),곤지암,골드,골프존카운티안성H,...,필로스,한림광릉,한림용인,한성,한양,한양파인(P9),한원,해솔리아,화성(P9),화성상록
cc_name,,,,,,,,,,,,,,,,,,,,,
360도,1.000000,0.127315,0.083301,0.0,0.0,0.080223,0.0,0.076589,0.089323,0.051565,...,0.119028,0.108239,0.022208,0.036934,0.020085,0.015109,0.063174,0.070750,0.000000,0.021922
88,0.127315,1.000000,0.121103,0.0,0.0,0.165174,0.0,0.046346,0.152532,0.088981,...,0.061556,0.137236,0.040934,0.153491,0.030233,0.049017,0.053266,0.031397,0.026264,0.045146
H1 CLUB(구 덕평),0.083301,0.121103,1.000000,0.0,0.0,0.106111,0.0,0.097707,0.120663,0.105919,...,0.082755,0.089859,0.054910,0.120163,0.043559,0.065348,0.053949,0.025347,0.037731,0.015382
가산노블리스,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
가평베네스트,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.074358,0.041324,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
한양파인(P9),0.015109,0.049017,0.065348,0.0,0.0,0.072396,0.0,0.000000,0.056063,0.033312,...,0.056003,0.047626,0.039003,0.014750,0.026684,1.000000,0.063751,0.000000,0.016251,0.042349
한원,0.063174,0.053266,0.053949,0.0,0.0,0.064732,0.0,0.000000,0.022292,0.040795,...,0.019949,0.028375,0.102120,0.058563,0.000000,0.063751,1.000000,0.043373,0.052212,0.000000
해솔리아,0.070750,0.031397,0.025347,0.0,0.0,0.042029,0.0,0.000000,0.087234,0.058366,...,0.025494,0.043039,0.044781,0.051965,0.000000,0.000000,0.043373,1.000000,0.021170,0.000000


In [31]:
# cc: 137개, user: 6108개
# train_df에 포함된 user id_num을 계산에 반영한다
userId_grouped = ratings_cc.groupby('id_num')
# index: id_num, columns: total cc_num
item_prediction_result_df = pd.DataFrame(index=list(userId_grouped.indices.keys()), columns=item_sparse_matrix.index)
item_prediction_result_df

cc_name,360도,88,H1 CLUB(구 덕평),가산노블리스,가평베네스트,강남300,고양컨트리클럽(P9),곤지암,골드,골프존카운티안성H,...,필로스,한림광릉,한림용인,한성,한양,한양파인(P9),한원,해솔리아,화성(P9),화성상록
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
for userId, group in tqdm(userId_grouped):
    # user가 rating한 cc_name * 전체 cc_name
    user_sim = item_cossim_df.loc[group['cc_name']]
    # user가 rating한 cc_name * 1
    user_rating = group['cc_score']
    # 전체 cc_name * 1
    sim_sum = user_sim.sum(axis=0)

    # userId의 전체 rating predictions (6108 * 1)
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    item_prediction_result_df.loc[userId] = pred_ratings

  0%|          | 0/6108 [00:00<?, ?it/s]

In [33]:
item_prediction_result_df

cc_name,360도,88,H1 CLUB(구 덕평),가산노블리스,가평베네스트,강남300,고양컨트리클럽(P9),곤지암,골드,골프존카운티안성H,...,필로스,한림광릉,한림용인,한성,한양,한양파인(P9),한원,해솔리아,화성(P9),화성상록
1,1.003954,1.432387,0.825223,0.0,0.454371,1.144316,0.0,0.0,0.959027,0.801813,...,0.839423,1.242538,0.418595,0.969129,0.552634,0.710348,0.483014,0.574779,0.496594,0.591782
2,1.873961,2.281312,1.819262,0.0,0.279146,2.094165,0.0,0.789521,2.272896,1.804871,...,1.514157,2.297985,0.869882,1.850747,0.453309,0.826471,0.906341,1.336531,0.473409,0.553037
3,0.948996,1.585248,1.233794,0.0,0.0,1.207191,0.0,0.0,1.73463,4.985101,...,0.753127,1.659819,0.604694,1.277155,0.55593,0.781057,0.852551,0.832288,0.491718,0.422473
4,1.228821,0.929876,0.944287,0.0,0.0,0.782795,0.0,0.0,0.801118,0.652985,...,0.75418,1.061037,0.43723,0.63311,0.04194,0.475331,0.669298,0.622957,0.251435,0.0
5,0.356246,0.500687,0.456787,0.0,0.160302,0.485161,0.0,0.0,0.439861,0.2903,...,0.61268,0.828813,0.210538,0.346566,0.229488,0.234204,0.177551,0.131509,0.088748,0.051556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,0.819986,1.323453,1.076711,0.0,0.0,1.204175,0.0,0.533564,5.0,0.979201,...,0.651911,1.20334,0.632934,0.748877,0.406955,0.53087,0.218055,0.802351,0.441684,0.108515
6105,1.08839,0.974983,0.827846,0.0,0.0,0.860554,0.0,0.0,1.070231,0.527388,...,0.963319,1.225921,0.577897,0.625432,0.08761,0.182357,0.356197,0.629858,0.506608,0.300753
6106,1.499331,2.418954,2.042315,0.0,0.91804,2.227619,0.0,0.0,2.098686,1.332751,...,1.637611,2.665188,0.931941,2.285172,0.38999,1.016908,1.337863,1.020972,1.171644,0.579665
6107,1.618556,2.172576,1.386937,0.0,0.987412,1.974042,0.0,0.783201,1.724157,1.270033,...,1.705956,2.188301,0.68035,1.494862,0.060064,0.911195,0.420515,0.816601,0.400026,0.70777


In [34]:
# 사용자가 평점을 부여한 골프장에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(item_prediction_result_df.values, sparse_matrix.fillna(0).values))

아이템 기반 모든 인접 이웃 MSE:  10.832479078229328


### top-n 유사도를 가진 데이터들에 대해서만 예측 평점 계산
- MSE 값을 감소하기 위해 특정 골프장과 가장 비슷한 유사도를 가지는 골프장에 대해서만 유사도 벡터를 적용

In [35]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=10):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))
    return pred

In [37]:
# 실행시간 2분 정도 걸림
item_rating_prediction = predict_rating_topsim(sparse_matrix.fillna(0).values, item_cossim_df.values, n=5)
print('아이템 기반 인접 TOP-5 이웃 MSE: ', get_mse(item_rating_prediction, sparse_matrix.fillna(0).values))

/var/folders/dz/4s3j51857zn3t537wl7_r8th0000gn/T/ipykernel_1051/971470633.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
/var/folders/dz/4s3j51857zn3t537wl7_r8th0000gn/T/ipykernel_1051/971470633.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))


아이템 기반 인접 TOP-5 이웃 MSE:  9.279283556985003


In [38]:
# 계산된 예측 평점 데이터는 DataFrame으로 재생성
item_rating_prediction_matrix = pd.DataFrame(data=item_rating_prediction, index= sparse_matrix.index, columns = sparse_matrix.columns)
item_rating_prediction_matrix

cc_name,360도,88,H1 CLUB(구 덕평),가산노블리스,가평베네스트,강남300,고양컨트리클럽(P9),곤지암,골드,골프존카운티안성H,...,필로스,한림광릉,한림용인,한성,한양,한양파인(P9),한원,해솔리아,화성(P9),화성상록
id_num,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,6.143574,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.905377,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.431260,0.568460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,5.553975,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6105,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6106,0.000000,0.0,0.0,0.0,0.000000,0.827672,0.000000,0.0,0.000000,0.000000,...,0.000000,0.829867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
# 추천에 앞서 2번 사용자가 높은 평점을 준 골프장을 확인
user_rating_id = sparse_matrix.loc[2, :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:5]

cc_name
아리지    10.0
금강      8.0
그린힐     7.5
Name: 2, dtype: float64

#### 사용자가 방문하지 않은 골프장 중에서 추천해보자
- user_rating이 0보다 크면 기존에 방문한 골프장이라는 점을 이용해서 계산

In [6]:
def get_unseen_cc(ratings_matrix, id_num):
    # id_num으로 입력받은 사용자의 모든 골프장 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 골프장명(cc_name)을 index로 가지는 Series 객체임. 
    user_rating = sparse_matrix.loc[id_num,:]
    
    # user_rating이 0보다 크면 기존에 방문한 골프장. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[user_rating > 0].index.tolist()
    
    # 모든 골프장을 list 객체로 만듬.
    cc_list = sparse_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 cc는 cc_list에서 제외함. 
    unseen_list = [cc for cc in cc_list if cc not in already_seen]
    
    return unseen_list

In [41]:
# pred_df : 앞서 계산된 골프장별 예측 평점
# unseen_list : 사용자가 방문하지 않은 골프장
# top_n : 상위 n개를 가져온다.

def recomm_cc_by_id(pred_df, id_num, unseen_list, top_n=5):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 골프장명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_cc = pred_df.loc[id_num, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_cc

In [43]:
# 사용자가 방문하지 않은 골프장 추출   
unseen_list = get_unseen_cc(sparse_matrix, 2)

# 아이템 기반의 인접 이웃 협업 필터링으로 골프장 추천 
recomm_10_cc = recomm_cc_by_id(item_prediction_result_df, 2, unseen_list, top_n=5)

# 평점 데이타를 DataFrame으로 생성. 
recomm_10_cc = pd.DataFrame(data=recomm_10_cc.values, index=recomm_10_cc.index, columns=['pred_score'])
recomm_10_cc

,pred_score
cc_name,
스카이밸리,2.648137
은화삼,2.570649
기흥,2.499166
안성,2.448192
블루헤런,2.384433


## CF - MF기반 잠재요인 CF

#### CC 총 평점 데이터 기준

In [7]:
# matrix 확인

print(ratings_cc_matrix.shape, ratings_caddie_matrix.shape, ratings_course_matrix.shape, ratings_price_matrix.shape, ratings_facility_matrix.shape, sep='\n') 

# print(ratings_matrix.shape)
# ratings_matrix.head()

(6108, 137)
(6108, 137)
(6108, 137)
(6108, 137)
(6108, 137)


In [8]:
def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]

    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [9]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0
       
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
   
    # P와 Q 매트릭스를 계속 업데이트(확률적 경사하강법)
    for step in tqdm(range(steps)):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
            
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
            
    return P, Q

In [37]:
%%time
# 경사하강법을 이용한 행렬 분해
P1, Q1 = matrix_factorization(ratings_cc_matrix.values, K=50, steps=120, learning_rate=0.0001, r_lambda = 0.01)
P2, Q2 = matrix_factorization(ratings_caddie_matrix.values, K=50, steps=120, learning_rate=0.0001, r_lambda = 0.01)
P3, Q3 = matrix_factorization(ratings_course_matrix.values, K=50, steps=120, learning_rate=0.0001, r_lambda = 0.01)
P4, Q4 = matrix_factorization(ratings_price_matrix.values, K=50, steps=120, learning_rate=0.0001, r_lambda = 0.01)
P5, Q5 = matrix_factorization(ratings_facility_matrix.values, K=50, steps=120, learning_rate=0.0001, r_lambda = 0.01)

pred_cc_matrix = np.dot(P1, Q1.T)
pred_caddie_matrix = np.dot(P2, Q2.T)
pred_course_matrix = np.dot(P3, Q3.T)
pred_price_matrix = np.dot(P4, Q4.T)
pred_facility_matrix = np.dot(P5, Q5.T)

  0%|          | 0/120 [00:00<?, ?it/s]

### iteration step :  0  rmse :  8.771357024188804
### iteration step :  10  rmse :  8.77099899650937
### iteration step :  20  rmse :  8.770572878950302
### iteration step :  30  rmse :  8.769975184647851
### iteration step :  40  rmse :  8.769004126391856
### iteration step :  50  rmse :  8.76722478564395
### iteration step :  60  rmse :  8.763660470561465
### iteration step :  70  rmse :  8.756082023316235
### iteration step :  80  rmse :  8.739396634147043
### iteration step :  90  rmse :  8.702161297752246
### iteration step :  100  rmse :  8.619905820863345
### iteration step :  110  rmse :  8.446329395677482


  0%|          | 0/120 [00:00<?, ?it/s]

### iteration step :  0  rmse :  9.053204790964777
### iteration step :  10  rmse :  9.052836195824025
### iteration step :  20  rmse :  9.052396059875475
### iteration step :  30  rmse :  9.051775408139669
### iteration step :  40  rmse :  9.050759833155315
### iteration step :  50  rmse :  9.048881624106327
### iteration step :  60  rmse :  9.045075916108626
### iteration step :  70  rmse :  9.036876658987966
### iteration step :  80  rmse :  9.018567689898857
### iteration step :  90  rmse :  8.977139583233765
### iteration step :  100  rmse :  8.884563438295189
### iteration step :  110  rmse :  8.688208817950485


  0%|          | 0/120 [00:00<?, ?it/s]

### iteration step :  0  rmse :  8.820599685697527
### iteration step :  10  rmse :  8.820238827716732
### iteration step :  20  rmse :  8.819807794044973
### iteration step :  30  rmse :  8.819199228493504
### iteration step :  40  rmse :  8.81820260828429
### iteration step :  50  rmse :  8.816361872003043
### iteration step :  60  rmse :  8.812648008054104
### iteration step :  70  rmse :  8.804702092299756
### iteration step :  80  rmse :  8.787114512740242
### iteration step :  90  rmse :  8.747704586200658
### iteration step :  100  rmse :  8.660479548967247
### iteration step :  110  rmse :  8.476892331179245


  0%|          | 0/120 [00:00<?, ?it/s]

### iteration step :  0  rmse :  8.8291223164856
### iteration step :  10  rmse :  8.828760648823804
### iteration step :  20  rmse :  8.828331409325807
### iteration step :  30  rmse :  8.827733267403532
### iteration step :  40  rmse :  8.82677119383078
### iteration step :  50  rmse :  8.825029382294645
### iteration step :  60  rmse :  8.821582501289083
### iteration step :  70  rmse :  8.814335016975631
### iteration step :  80  rmse :  8.798531201959333
### iteration step :  90  rmse :  8.763539897198124
### iteration step :  100  rmse :  8.68667103636271
### iteration step :  110  rmse :  8.524742054177661


  0%|          | 0/120 [00:00<?, ?it/s]

### iteration step :  0  rmse :  8.631488762879158
### iteration step :  10  rmse :  8.631136609309744
### iteration step :  20  rmse :  8.63072130642258
### iteration step :  30  rmse :  8.630148601595542
### iteration step :  40  rmse :  8.6292398606202
### iteration step :  50  rmse :  8.627620587624795
### iteration step :  60  rmse :  8.624471552203001
### iteration step :  70  rmse :  8.617968811269327
### iteration step :  80  rmse :  8.604039295833866
### iteration step :  90  rmse :  8.573700125764857
### iteration step :  100  rmse :  8.507903856566813
### iteration step :  110  rmse :  8.36997803664623
CPU times: user 4min 36s, sys: 3min 32s, total: 8min 8s
Wall time: 1min 18s


In [38]:
ratings_pred_cc_matrix = pd.DataFrame(data=pred_cc_matrix, index= ratings_cc_matrix.index, columns = ratings_cc_matrix.columns)
ratings_pred_caddie_matrix = pd.DataFrame(data=pred_caddie_matrix, index= ratings_caddie_matrix.index, columns = ratings_caddie_matrix.columns)
ratings_pred_course_matrix = pd.DataFrame(data=pred_course_matrix, index= ratings_course_matrix.index, columns = ratings_course_matrix.columns)
ratings_pred_price_matrix = pd.DataFrame(data=pred_price_matrix, index= ratings_price_matrix.index, columns = ratings_price_matrix.columns)
ratings_pred_facility_matrix = pd.DataFrame(data=pred_facility_matrix, index= ratings_facility_matrix.index, columns = ratings_facility_matrix.columns)

In [ ]:
# ratings_pred_cc_matrix = pd.DataFrame(data=pred_cc_matrix, index= ratings_cc_matrix.index, columns = ratings_cc_matrix.columns)
# # 예측 평점 행렬 확인
# print(ratings_pred_cc_matrix.shape)
# ratings_pred_matrix.head(15)

In [ ]:
# # 원본 행렬 확인
# print(ratings_matrix.shape)
# ratings_matrix.head(15)

### 2번 사용자에게 예측 평점이 높은 골프장 추천

In [39]:
# userId의 카테고리별 예측 평점
def recommendation_golf(userId):
    feature_list = ['cc', 'caddie', 'course', 'price', 'facility']
    matrices_list = ['ratings_'+ feature_list[i] + '_matrix' for i in range(len(feature_list))]
    pred_matrices_list = ['ratings_pred_'+ feature_list[i] + '_matrix' for i in range(len(feature_list))]
    for i in range(len(matrices_list)):
        cc_list = get_unseen_cc(globals()[matrices_list[i]], userId)
        recomm_10_cc = recomm_cc_by_id(globals()[pred_matrices_list[i]], userId, cc_list, top_n=10)
        recomm_10_cc = pd.DataFrame(data=recomm_10_cc.values, index=recomm_10_cc.index, columns=['pred_score'])
        print(pred_matrices_list[i])
        display(recomm_10_cc)

In [48]:
recommendation_golf(936)

ratings_pred_cc_matrix


,pred_score
cc_name,
스카이밸리,0.136264
기흥,0.121786
금강,0.113588
88,0.096647
캐슬파인,0.081940
은화삼,0.079766
한림광릉,0.078910
골드,0.071482
안성베네스트,0.071167


ratings_pred_caddie_matrix


,pred_score
cc_name,
스카이밸리,0.153684
기흥,0.137256
금강,0.127102
88,0.107937
한림광릉,0.098049
캐슬파인,0.096727
은화삼,0.092397
골드,0.085402
안성베네스트,0.084945


ratings_pred_course_matrix


,pred_score
cc_name,
스카이밸리,0.145693
기흥,0.130743
금강,0.118787
88,0.101656
은화삼,0.081547
캐슬파인,0.075739
안성베네스트,0.075411
태광,0.073353
한림광릉,0.071463


ratings_pred_price_matrix


,pred_score
cc_name,
스카이밸리,0.125308
기흥,0.115757
금강,0.108362
88,0.089829
캐슬파인,0.082591
은화삼,0.077488
한림광릉,0.075411
골드,0.070188
안성베네스트,0.062841


ratings_pred_facility_matrix


,pred_score
cc_name,
스카이밸리,0.115041
기흥,0.102862
금강,0.097166
88,0.083616
캐슬파인,0.071964
한림광릉,0.070480
은화삼,0.064379
태광,0.060895
뉴서울,0.060885


In [40]:
def check_recommendation_golf(userId):
    feature_list = ['cc', 'caddie', 'course', 'price', 'facility']
    matrices_list = ['ratings_'+ feature_list[i] + '_matrix' for i in range(len(feature_list))]
    pred_matrices_list = ['ratings_pred_'+ feature_list[i] + '_matrix' for i in range(len(feature_list))]
    for i in range(len(matrices_list)):
        cc_list = get_unseen_cc(globals()[matrices_list[i]], userId)
        recomm_10_cc = recomm_cc_by_id(globals()[pred_matrices_list[i]], userId, cc_list, top_n=10)
        recomm_10_cc = pd.DataFrame(data=recomm_10_cc.values, index=recomm_10_cc.index, columns=['pred_score'])
        if len(recomm_10_cc[recomm_10_cc['pred_score']>=10]) > 0:
            print(pred_matrices_list[i])
            display(len(recomm_10_cc[recomm_10_cc['pred_score']>=10]))

In [41]:
[check_recommendation_golf(i) for i in list(ratings_cc_matrix.index)]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [74]:
feature_list = ['cc', 'caddie', 'course', 'price', 'facility']

def recommendation_golf(userId):
    for i in range(feature_list):
        cc_list = get_unseen_cc('ratings_'+ feature_list[i] + '_matrix', 2)
        recomm_10_cc = recomm_cc_by_id('ratings_pred_'+ feature_list[i] +'_matrix', 2, cc_list, top_n=10)
        recomm_10_cc = pd.DataFrame(data=recomm_10_cc.values, index=recomm_10_cc.index, columns=['pred_score'])
        display(recomm_10_cc)

# 사용자가 방문하지 않은 골프장 추출   
unseen_list = get_unseen_cc(ratings_cc_matrix, 2)
# 사용자가 캐디 평가하지 않은 골프장 추출   
no_rate_caddie_list = get_unseen_cc(ratings_caddie_matrix, 2)
# 사용자가 캐디 평가하지 않은 골프장 추출   
no_rate_caddie_list = get_unseen_cc(ratings_caddie_matrix, 2)
# 사용자가 캐디 평가하지 않은 골프장 추출   
no_rate_caddie_list = get_unseen_cc(ratings_caddie_matrix, 2)
# 사용자가 캐디 평가하지 않은 골프장 추출   
no_rate_caddie_list = get_unseen_cc(ratings_caddie_matrix, 2)

# 아이템 기반의 인접 이웃 협업 필터링으로 골프장 추천 
recomm_10_cc = recomm_cc_by_id(ratings_pred_cc_matrix, 2, unseen_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_10_cc = pd.DataFrame(data=recomm_10_cc.values, index=recomm_10_cc.index, columns=['pred_score'])
display(recomm_10_cc)

,pred_score
cc_name,
자유,9.013458
안성베네스트,8.794760
아시아나,8.759057
솔모로,8.758710
소피아그린,8.746524
페럼,8.713490
푸른솔포천,8.676253
여주신라,8.674424
기흥,8.673011


In [73]:
# 사용자가 캐디 평가하지 않은 골프장 추출   
no_rate_caddie_list = get_unseen_cc(ratings_caddie_matrix, 2)

# 아이템 기반의 인접 이웃 협업 필터링으로 골프장 추천
recomm_10_caddie = recomm_cc_by_id(ratings_pred_caddie_matrix, 2, no_rate_caddie_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_10_caddie = pd.DataFrame(data=recomm_10_caddie.values, index=recomm_10_caddie.index, columns=['pred_score'])
recomm_10_caddie

,pred_score
cc_name,
기흥,9.987842
한성,9.883517
골프존카운티안성H,9.875972
글렌로스(P9),9.849335
자유,9.808933
레이크사이드,9.802457
서서울,9.801446
사우스스프링스,9.779068
소피아그린,9.750022
